In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

데이터 불러오고 데이터의 분포 확인하기

In [ ]:
# 데이터 불러오기
train_df=pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
test_df=pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
# 데이터 차원
print("데이터 차원 : ",train_df.shape)
print('\n')
# 데이터 보기
train_df.head()

In [ ]:
# 불균형한 데이터라는것을 알수있음
train_df['TARGET'].plot.hist()

결측값 확인

In [ ]:
def missing_values_table(df):
    # 전체 결측치 개수 확인
    mis_val=df.isnull().sum()
    
    # 결측치 비중 확인
    mis_val_percent=100*mis_val/len(df)
    
    # 결측치 개수 , 결측치 비중 테이블 만들기
    mis_val_table=pd.concat([mis_val, mis_val_percent],axis=1)
    
    # 컬럼 이름바꾸기
    mis_val_table_ren_columns=mis_val_table.rename(columns={0:'Missing Values',1:'% of Total Values'})

    # 결측치 0인 컬럼은 제외하고 정렬
    mis_val_table_ren_columns=mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1]!=0].sort_values('% of Total Values',ascending=False).round(1)

    # 요약 결과 print
    print("train_df의 전체 컬럼 개수는 "+str(df.shape[1])+"개 이다.\n"
         "그 중에서 결측치가 있는 컬럼 개수는 "+str(mis_val_table_ren_columns.shape[0])+'개 이다.')
    
    return mis_val_table_ren_columns

In [ ]:
# 결측값을 제거하거나 대체하기에 너무 개수가 많아 결과에 영향을 미칠거같다는 생각이 들어 일단은 냅두기로생각
missing_values=missing_values_table(train_df)
missing_values.head(20)

In [ ]:
# 라벨인코더
le=LabelEncoder()
for col in train_df:
    if(train_df[col].dtype=='object' and len(list(train_df[col].unique()))<=2):
        le.fit(train_df[col])
        train_df[col]=le.transform(train_df[col])
        test_df[col]=le.transform(test_df[col])

In [ ]:
# 원핫 인코더(get_dummies)
# 3개 이상의 value의 종류가 있을때 라벨인코딩을 하면 value 서로가 관계성을 갖을지도 모르기 때문에
# get_dummies로 원-핫 인코딩 진행
train_df=pd.get_dummies(train_df)
test_df=pd.get_dummies(test_df)

바꾸기

In [ ]:
# 서로 동일한 feature들로 구성하기위해 align(join='inner')로 교집합을 구해준다.
# TARGET변수는 train데이터에만 있지만 필요한 변수이기때문에 따로 빼두고나서 다시추가할것
train_labels=train_df['TARGET']

train_df,test_df=train_df.align(test_df,join='inner',axis=1)

# TARGET변수 다시 추가
train_df['TARGET']=train_labels

print('Training Features shape: ', train_df.shape)
print('Testing Features shape: ', test_df.shape)

In [ ]:
#(get_dummies로 인해 열의 개수 늘음)
train_df.shape

> TARGET 과 다른 피쳐사이의 관계성 보기

In [ ]:
# 피쳐종류 확인
train_df.columns.tolist()

In [ ]:
# 임의로 관련있어 보이는것들 확인해보기
train_df[['AMT_INCOME_TOTAL','CNT_CHILDREN','DAYS_BIRTH','DAYS_EMPLOYED']].describe()

In [ ]:
# from collections import Counter
# def detect_outliers(df,n,features):
#     outlier_indicers=[]
#     for col in features:
#         q1=np.percentile(df[col],25)
#         q3=np.percentile(df[col],75)
#         iqr=q3-q1
        
#         outlier_step=1.5*iqr
        
#         outlier_list_col=df[(df[col]<q1-outlier_step)|(df[col]>q3+outlier_step)].index
#         outlier_indicers.extend(outlier_list_col)
#     outlier_indicers=Counter(outlier_indicers)
#     multiple_outliers=list(k for k,v in outlier_indicers.items() if v>n)
    
#     return multiple_outliers
# outliers_to_drop=detect_outliers(train_df,2,['CNT_CHILDREN'])

In [ ]:
plt.figure(figsize=(10,10))
for i, source in enumerate(['AMT_INCOME_TOTAL','CNT_CHILDREN','DAYS_BIRTH','DAYS_EMPLOYED']):    
    plt.subplot(4,1,i+1)
    train_df[source].plot.hist()
    plt.xlabel(source)
plt.tight_layout(h_pad=2.5)

'DAYS_BIRTH'가 골고루 잘 분포되어있어 TARGET과 비교하기 적절할것같다.

'DATS_EMPLOYED'의 이상치가 있어 처리한후 다시 그래프를 봐보자

In [ ]:
# 이상치
anom=train_df[train_df['DAYS_EMPLOYED']==365243]
# 이상치 외
non_anom=train_df[train_df['DAYS_EMPLOYED']!=365243]

print('이상치 아닌값이 못갚는 비율 :  %0.2f%%' %(100*non_anom['TARGET'].mean()))
print('이상치값이 못갚는 비율 :  %0.2f%%' % (100 * anom['TARGET'].mean()))
print('못갚는 사람의 명수 %d' % len(anom))

In [ ]:
#일을 시작한지 얼마 안됬을수록 못갚을 확률이 높다.
train_df['DAYS_EMPLOYED_ANOM']=train_df['DAYS_EMPLOYED']==365243
train_df['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)
(train_df['DAYS_EMPLOYED']/-360).plot.hist()

In [ ]:
plt.figure(figsize=(5,5))
sns.kdeplot(train_df.loc[train_df['TARGET']==0,'DAYS_BIRTH']/-365,label='TARGET==0',color='red')
sns.kdeplot(train_df.loc[train_df['TARGET']==1,'DAYS_BIRTH']/-365,label='TARGET==1',color='green')

In [ ]:
plt.figure(figsize=(5,5))
sns.kdeplot(train_df.loc[train_df['TARGET']==0,'DAYS_EMPLOYED']/-365,label='TARGET==0',color='red')
sns.kdeplot(train_df.loc[train_df['TARGET']==1,'DAYS_EMPLOYED']/-365,label='TARGET==1',color='green')

일을 한지 얼마 안됬을수록, 나이가 적을수록 상환을 안하는 비율이 높다는것을 볼수 있다(TARGET==1).

In [ ]:
# TARGET과 FEATURE들의 상관계수 비교
# +이면 값이 커질수록 상환못할 가능성이 높다.(1)
# -이면 값이 커질수록 상환할 가능성이 높다.(0)
cor=train_df.corr()['TARGET'].sort_values()
print(cor.head(5))
print('\n')
print(cor.tail(5))

DAYS_BIRTH,DAYS_EMPLOYED 가 오래될수록 상환못할 가능성이 높은건 아마 value가 음수이기때문이라 생각이 드니
절댓값을 취해 다시 상관계수를 보자

In [ ]:
# DAYS_BIRTH의 절대값 취한 후 TARGET변수와의 상관계수
train_df['DAYS_BIRTH']=abs(train_df['DAYS_BIRTH'])
train_df['DAYS_EMPLOYED']=abs(train_df['DAYS_EMPLOYED'])

cor=train_df.corr()['TARGET'].sort_values()
print(cor.head(5))
print('\n')
print(cor.tail(5))

In [ ]:
# TARGET과의 상관계수가 음인것들 사이의 상관계수 비교 
cor_mdata=train_df[['TARGET','DAYS_EMPLOYED','DAYS_BIRTH','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]

plt.figure(figsize=(8,6))
sns.heatmap(data=cor_mdata.corr(),annot=True,cmap='Blues')
plt.title('Correlation Heatmap');

In [ ]:
# 모든 데이터의 상관계수(어...)
cor_mdata=train_df[train_df.columns]

plt.figure(figsize=(8,6))
sns.heatmap(data=cor_mdata.corr(),annot=True,cmap='Blues')
plt.title('Correlation Heatmap');

In [ ]:
# 'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3' 의 'TARGET'와의 경향 KDE그래프로 확인
plt.figure(figsize=(10,10))
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(train_df.loc[train_df['TARGET']==0,source],label='target==0')
    sns.kdeplot(train_df.loc[train_df['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
    
plt.tight_layout(h_pad=2.5)